In [ ]:
#ID+类别型变量

In [ ]:
sub = df.groupby(['EID', 'cate']).size().reset_index()    #第一个放id，第二个放要统计的类别特征
sub = sub.groupby('EID')[0].agg([sum, len]).reset_index() #计算出每个id一共有多少条记录和类别的种类
sub.columns = ['EID', 'cate', 'cate_types_count']   
sub_oh = df.groupby(['EID', 'cate']).size().unstack().reset_index()
sub_oh.columns = [i if i == 'EID' else 'cate_' + i for i in rec_wz_oh.columns]#计算每个类别的记录数

In [ ]:
#ID+时间

In [ ]:
df['recdate'] = pd.to_datetime(df['RECDATE']).apply(lambda x: 12*(x.year - 2010) + x.month - 1).fillna(
        -999).astype(int).replace(-999, np.NaN)#把日期按月转化为一个具体数字
rec_date = df.groupby('EID')['recdate'].agg([max, min, np.ptp, np.std]).reset_index()#每个id时间的方差均值极差标准差
rec_date.columns = ['EID', 'rec_date_max', 'rec_date_min', 'rec_date_ptp', 'rec_date_std']

In [ ]:
#分别计算与均值/最大/最小/和 的差距

In [ ]:
for column in ['MPNUM', 'INUM', 'FINZB', 'FSTINUM', 'TZINUM', 'ENUM', 'ZCZB', 'allnum', 'RGYEAR', 'alt_count', 'rig_count']:
    groupby_list = [['HY'], ['ETYPE'], ['HY', 'ETYPE'], ['HY', 'PROV'], ['ETYPE', 'PROV'], ['MPNUM_CLASS'], ['FSTINUM_CLASS']]
    for groupby in groupby_list:
        if 'MPNUM_CLASS' in groupby and column == 'MPNUM':
            continue
        if 'FSTINUM_CLASS' in groupby and column == 'FSTINUM':
            continue
        groupby_keylist = []
        for key in groupby:
            groupby_keylist.append(dataset[key])
        tmp = dataset[column].groupby(groupby_keylist).agg([sum, min, max, np.mean]).reset_index()
        tmp = pd.merge(dataset, tmp, on=groupby, how='left')
        dataset['ent_' + column.lower() + '-mean_gb_' + '_'.join(groupby).lower()] = dataset[column] - tmp['mean']
        dataset['ent_' + column.lower() + '-min_gb_' + '_'.join(groupby).lower()] = dataset[column] - tmp['min']
        dataset['ent_' + column.lower() + '-max_gb_' + '_'.join(groupby).lower()] = dataset[column] - tmp['max']
        dataset['ent_' + column.lower() + '/sum_gb_' + '_'.join(groupby).lower()] = dataset[column] / tmp['sum']
dataset.drop(['MPNUM_CLASS', 'FSTINUM_CLASS'], axis=1, inplace=True)

In [ ]:
#one_hot编码之后再特征交叉
dataset = dataset.join(pd.get_dummies(dataset['PROV'], prefix='prov'))
dataset = dataset.join(pd.get_dummies(dataset['HY'], prefix='hy').mul(dataset['ZCZB'], axis=0))
dataset = dataset.join(pd.get_dummies(dataset['ETYPE'], prefix='etype').mul(dataset['RGYEAR'], axis=0))
dataset.drop(['PROV', 'HY', 'ETYPE'], axis=1, inplace=True)